## Step 1: Import Libraries

In [1]:
# import necessary libraries

import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime
import glob
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables
load_dotenv()

True

## Step 2: Define Path

In [2]:
# Configuration - Use correct path for notebooks directory

DATA_PATH = "../data/raw/"
OUTPUT_PATH = "../data/processed/"
NOAA_TOKEN = os.getenv('NOAA_TOKEN')

# Display project information
project_info = {
    "project_name": "NYC CitiBike Demand Analysis 2022",
    "data_source": "Citi Bike System Data",
    "year": 2022,
    "weather_station": "LaGuardia Airport (NYC)",
    "station_id": "GHCND:USW00014732"
}

markdown_content = f"""
# {project_info['project_name']}

## Project Overview
- **Data Source**: {project_info['data_source']}
- **Analysis Period**: {project_info['year']}
- **Weather Station**: {project_info['weather_station']}
- **Station ID**: {project_info['station_id']}

## Analysis Objectives
1. Identify peak demand periods and popular stations
2. Analyze seasonal patterns and weather impact
3. Optimize bike distribution across NYC
4. Identify expansion opportunities
"""

display(Markdown(markdown_content))
print(f"NOAA Token: {'Loaded' if NOAA_TOKEN else 'NOT FOUND - Check .env file'}")


# NYC CitiBike Demand Analysis 2022

## Project Overview
- **Data Source**: Citi Bike System Data
- **Analysis Period**: 2022
- **Weather Station**: LaGuardia Airport (NYC)
- **Station ID**: GHCND:USW00014732

## Analysis Objectives
1. Identify peak demand periods and popular stations
2. Analyze seasonal patterns and weather impact
3. Optimize bike distribution across NYC
4. Identify expansion opportunities


NOAA Token: Loaded


## Step 3: Load CtiBike Data

In [ ]:
# Load CitiBike Data

print("Loading CitiBike data files...")

print(f"Using data path: {DATA_PATH}")
print(f"Path exists: {os.path.exists(DATA_PATH)}")

# Get all CSV files
all_files = []
for item in os.listdir(DATA_PATH):
    if item.endswith('.csv') and '2022' in item and 'citibike' in item:
        full_path = os.path.join(DATA_PATH, item)
        all_files.append(full_path)

all_files = sorted(all_files)
file_count = len(all_files)

print(f"Found {file_count} CSV files to process")

# Display first few files
print("First 5 files:")
for i, file_path in enumerate(all_files[:5], 1):
    file_name = os.path.basename(file_path)
    file_size = round(os.path.getsize(file_path) / (1024*1024), 2)
    print(f"  {i}. {file_name} ({file_size} MB)")

def read_citibike_file(file_path):
    try:
        file_name = os.path.basename(file_path)
        file_size = os.path.getsize(file_path) / (1024*1024)  # MB
        print(f"Reading: {file_name} ({file_size:.1f} MB)")
        df = pd.read_csv(file_path, low_memory=False)
        df['_source_file'] = file_name
        print(f"  Success: {len(df):,} rows, {len(df.columns)} columns")
        return df
    except Exception as e:
        print(f"  ERROR reading {file_path}: {e}")
        return None

print("\nStarting to load files...")
dataframes = []
for file_path in all_files:
    df = read_citibike_file(file_path)
    if df is not None:
        dataframes.append(df)

if dataframes:
    df_bikes = pd.concat(dataframes, ignore_index=True)
    print(f"\n=== SUCCESS ===")
    print(f"Successfully concatenated {len(dataframes)} files")
    print(f"Total dataset: {len(df_bikes):,} rows, {len(df_bikes.columns)} columns")
    
    # Display dataset info
    print(f"\nDate range:")
    # Find datetime columns
    datetime_cols = [col for col in df_bikes.columns if 'time' in col.lower() or 'date' in col.lower()]
    if datetime_cols:
        for col in datetime_cols[:2]:  # Check first 2 datetime columns
            if col in df_bikes.columns:
                print(f"  {col}: {df_bikes[col].min()} to {df_bikes[col].max()}")
    
    print(f"\nFirst 5 rows:")
    display(df_bikes.head(5))
    
else:
    raise Exception("No data files were successfully loaded")

Loading CitiBike data files...
Using data path: ../data/raw/
Path exists: True
Found 38 CSV files to process
First 5 files:
  1. 202201-citibike-tripdata_1.csv (185.56 MB)
  2. 202201-citibike-tripdata_2.csv (4.5 MB)
  3. 202202-citibike-tripdata_1.csv (185.96 MB)
  4. 202202-citibike-tripdata_2.csv (36.84 MB)
  5. 202203-citibike-tripdata_1.csv (186.27 MB)

Starting to load files...
Reading: 202201-citibike-tripdata_1.csv (185.6 MB)
  Success: 1,000,000 rows, 14 columns
Reading: 202201-citibike-tripdata_2.csv (4.5 MB)
  Success: 24,555 rows, 14 columns
Reading: 202202-citibike-tripdata_1.csv (186.0 MB)
  Success: 1,000,000 rows, 14 columns
Reading: 202202-citibike-tripdata_2.csv (36.8 MB)
  Success: 197,312 rows, 14 columns
Reading: 202203-citibike-tripdata_1.csv (186.3 MB)
  Success: 1,000,000 rows, 14 columns
Reading: 202203-citibike-tripdata_2.csv (157.5 MB)
  Success: 845,965 rows, 14 columns
Reading: 202204-citibike-tripdata_1.csv (186.4 MB)
  Success: 1,000,000 rows, 14 columns


## Step 4: Data Cleaning and Preprocessing

In [ ]:
# Data Cleaning & Preprocessing

print("Starting data cleaning and preprocessing...")

# Convert datetime columns
print("Converting datetime columns...")
df_bikes['started_at'] = pd.to_datetime(df_bikes['started_at'], errors='coerce')
df_bikes['ended_at'] = pd.to_datetime(df_bikes['ended_at'], errors='coerce')

# Extract date for weather merging
df_bikes['date'] = df_bikes['started_at'].dt.date
df_bikes['date'] = pd.to_datetime(df_bikes['date'])

# Data quality check
initial_rows = len(df_bikes)
df_bikes = df_bikes.dropna(subset=['started_at', 'ended_at'])
final_rows = len(df_bikes)
removed_rows = initial_rows - final_rows

print(f"Data cleaning complete:")
print(f"  - Initial rows: {initial_rows:,}")
print(f"  - Rows removed (invalid dates): {removed_rows:,}")
print(f"  - Final rows: {final_rows:,}")
print(f"  - Data retention: {round((final_rows/initial_rows)*100, 2)}%")

# Display date range
print(f"\nDate range:")
print(f"  - Earliest trip: {df_bikes['started_at'].min()}")
print(f"  - Latest trip: {df_bikes['started_at'].max()}")
print(f"  - Total days: {df_bikes['date'].nunique()}")

# Display basic statistics
print(f"\nBasic statistics:")
print(f"  - Member vs Casual:")
print(f"    {df_bikes['member_casual'].value_counts().to_dict()}")
print(f"  - Bike types:")
print(f"    {df_bikes['rideable_type'].value_counts().to_dict()}")

## Step 5: Fetch Weather Data

In [ ]:
# Fetch Weather Data from NOAA API

print("Fetching weather data from NOAA API...")

station_id = "GHCND:USW00014732"  # LaGuardia Airport, NYC
start_date = "2022-01-01"
end_date = "2022-12-31"

noaa_url = (
    f"https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"
    f"datasetid=GHCND&datatypeid=TAVG&limit=1000&"
    f"stationid={station_id}&startdate={start_date}&enddate={end_date}"
)

try:
    headers = {'token': NOAA_TOKEN}
    response = requests.get(noaa_url, headers=headers)
    
    if response.status_code == 200:
        weather_data = json.loads(response.text)
        
        if 'results' in weather_data and weather_data['results']:
            # Extract temperature data
            avg_temps = [item for item in weather_data['results'] if item['datatype'] == 'TAVG']
            dates_temp = [item['date'] for item in avg_temps]
            temps = [item['value'] for item in avg_temps]
            
            # Create weather dataframe
            df_weather = pd.DataFrame()
            df_weather['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
            df_weather['avg_temp_c'] = [float(v) / 10.0 for v in temps]  # Convert to Celsius
            df_weather['avg_temp_f'] = df_weather['avg_temp_c'] * 9/5 + 32  # Convert to Fahrenheit
            
            print(f"Successfully retrieved {len(df_weather)} days of weather data")
            
            # Display weather summary
            print(f"\nWeather data summary:")
            print(f"  - Date range: {df_weather['date'].min().strftime('%Y-%m-%d')} to {df_weather['date'].max().strftime('%Y-%m-%d')}")
            print(f"  - Average temperature: {df_weather['avg_temp_c'].mean():.1f}°C ({df_weather['avg_temp_f'].mean():.1f}°F)")
            print(f"  - Min temperature: {df_weather['avg_temp_c'].min():.1f}°C")
            print(f"  - Max temperature: {df_weather['avg_temp_c'].max():.1f}°C")
            
        else:
            print("No weather data returned from API")
            df_weather = pd.DataFrame(columns=['date', 'avg_temp_c', 'avg_temp_f'])
            
    else:
        print(f"API request failed with status code: {response.status_code}")
        df_weather = pd.DataFrame(columns=['date', 'avg_temp_c', 'avg_temp_f'])
        
except Exception as e:
    print(f"Error fetching weather data: {e}")
    df_weather = pd.DataFrame(columns=['date', 'avg_temp_c', 'avg_temp_f'])

# Display sample of weather data
if not df_weather.empty:
    print(f"\nSample of weather data:")
    display(df_weather.head())
else:
    print("No weather data available")

## Step 6: Merge Datastes

In [ ]:
# Merge Bike Data with Weather Data

print("Merging bike data with weather data...")

if not df_weather.empty:
    df_merged = df_bikes.merge(df_weather, how='left', on='date', indicator=True)
    
    # Analyze merge results
    merge_stats = df_merged['_merge'].value_counts()
    
    print(f"Merge completed:")
    print(f"  - Both (successful matches): {merge_stats.get('both', 0):,} rows")
    print(f"  - Left only (no weather data): {merge_stats.get('left_only', 0):,} rows")
    print(f"  - Right only (extra weather data): {merge_stats.get('right_only', 0):,} rows")
    print(f"  - Merge success rate: {round((merge_stats.get('both', 0) / len(df_merged)) * 100, 2)}%")
    
    # Remove merge indicator column
    df_merged = df_merged.drop('_merge', axis=1)
    
else:
    df_merged = df_bikes.copy()
    df_merged['avg_temp_c'] = np.nan
    df_merged['avg_temp_f'] = np.nan
    print("No weather data available - proceeding with NaN values for temperature")

print(f"\nFinal merged dataset:")
print(f"  - Total rows: {len(df_merged):,}")
print(f"  - Total columns: {len(df_merged.columns)}")
print(f"  - Memory usage: {round(df_merged.memory_usage(deep=True).sum() / (1024**3), 2)} GB")

# Display final dataset structure
print(f"\nFinal dataset columns:")
for col in df_merged.columns:
    print(f"  - {col}")

## Step 7: Export Processed Data

In [ ]:
# Export Processed Data

print("Exporting processed data...")

output_file = os.path.join(OUTPUT_PATH, "nyc_citibike_2022_processed.csv")
print(f"Output file: {output_file}")

# Save to CSV
df_merged.to_csv(output_file, index=False)

# Verify file was created
if os.path.exists(output_file):
    file_size = round(os.path.getsize(output_file) / (1024**3), 2)
    print(f"Export completed successfully:")
    print(f"  - File size: {file_size} GB")
    print(f"  - Total rows: {len(df_merged):,}")
    print(f"  - Total columns: {len(df_merged.columns)}")
    print(f"  - Export timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Display sample of exported data
    print(f"\nSample of exported data:")
    sample_cols = ['started_at', 'ended_at', 'start_station_name', 'end_station_name', 'member_casual', 'avg_temp_c']
    available_cols = [col for col in sample_cols if col in df_merged.columns]
    display(df_merged[available_cols].head(5))
    
else:
    print("Error: Output file was not created successfully")
    raise Exception("Data export failed")

## Step 8: Project Summary

In [ ]:
# Project Summary
print("Generating project summary...")

summary_stats = {
    'total_trips': len(df_merged),
    'date_range_days': df_merged['date'].nunique(),
    'data_files_processed': len(dataframes),
    'weather_days_integrated': len(df_weather) if not df_weather.empty else 0,
    'memory_usage_gb': round(df_merged.memory_usage(deep=True).sum() / (1024**3), 2),
    'completion_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

# Calculate additional statistics
user_types = df_merged['member_casual'].value_counts()
bike_types = df_merged['rideable_type'].value_counts()

markdown_content = f"""
# Data Collection Phase Complete

## Project Summary

### Data Processing Results
- **Total Bike Trips Processed**: {summary_stats['total_trips']:,}
- **Data Files Consolidated**: {summary_stats['data_files_processed']}
- **Days in Dataset**: {summary_stats['date_range_days']}
- **Weather Data Points**: {summary_stats['weather_days_integrated']}
- **Final Dataset Size**: {summary_stats['memory_usage_gb']} GB

### User Statistics
- **Member Rides**: {user_types.get('member', 0):,}
- **Casual Rides**: {user_types.get('casual', 0):,}

### Bike Type Distribution
"""

for bike_type, count in bike_types.items():
    markdown_content += f"- **{bike_type}**: {count:,}\n"

display(Markdown(markdown_content))